In [1]:
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from rdkit.Chem import rdMolAlign
from rdkit.ML.Cluster import Butina
from collections import defaultdict
import rdkit
import glob
import os
print(rdkit.__version__)

2023.09.6


In [2]:
cluster_type="RMSD"
logger=None
threshold=1.5

In [3]:
import py3Dmol
view = py3Dmol.view(query='pdb:1ubq')
view.setStyle({'cartoon':{'color':'spectrum'}})
view

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
import py3Dmol
def drawit(m, cids=[-1], p=None, removeHs=True,
           colors=('cyanCarbon','redCarbon','blueCarbon','magentaCarbon','whiteCarbon','purpleCarbon')):
        if removeHs:
            m = Chem.RemoveHs(m)
        if p is None:
            p = py3Dmol.view(width=400, height=400)
        p.removeAllModels()
        for i,cid in enumerate(cids):
            IPythonConsole.addMolToView(m,p,confId=cid)
        for i,cid in enumerate(cids):
            p.setStyle({'model':i,},
                            {'line':{'colorscheme':colors[i%len(colors)]}})
        p.zoomTo()
        return p.show()

In [5]:
    id_to_file_dict = defaultdict()
    extension = "pdb"
    list_files = sorted(glob.glob(os.path.join("./01-PDBs_OUT", "*.{}".format(extension))))
    nfiles = len(list_files)
    if extension == "mol2":
        molrdkit = Chem.MolFromMol2File(list_files[0], removeHs=False)
    elif extension == "pdb":
        molrdkit = Chem.MolFromPDBFile(list_files[0], removeHs=False)
    else:
        m = '\n\t\t ERROR: "{}" files are not still supported in clusterize rdkit.\n'.format(extension)
        print(m) if logger is None else logger.error(m)
        exit()
    id_to_file_dict[0] = list_files[0]

    nconf = 1
    for iconf in range(1, nfiles):

        if extension == "mol2":
            mtmp = Chem.MolFromMol2File(list_files[iconf], removeHs=False)
        elif extension == "pdb":
            mtmp = Chem.MolFromPDBFile(list_files[iconf], removeHs=False)
        else:
            m = "\n\t\t ERROR: {} files are not still soported\n".format(extension)
            print(m) if logger is None else logger.error(m)
            exit()
        conf = Chem.Conformer(mtmp.GetConformer(0))
        molrdkit.AddConformer(conf, assignId=True)
        id_to_file_dict[nconf] = list_files[iconf]
        nconf += 1

In [6]:
    # Remove Hydrogens ==================================================================
    m = "\t\t Number of initial conformers: {}\n".format(molrdkit.GetNumConformers())
    m += "\t\t Total Number of atoms: {}\n".format(molrdkit.GetNumAtoms())
    molrdkit_no_h = Chem.RemoveHs(molrdkit)
    m += "\t\t Remove Hydrogens to perform the clustering\n"
    m += "\t\t Heavy Number of atoms: {}\n".format(molrdkit_no_h.GetNumAtoms())
    print(m) if logger is None else logger.info(m)

		 Number of initial conformers: 5
		 Total Number of atoms: 32
		 Remove Hydrogens to perform the clustering
		 Heavy Number of atoms: 12



In [7]:
    # Find a pair of conformers with a decent size mismatch
    # between the direct alignment (which does not take symmetry into account) and the best alignment (which does):
    maxd = -100
    for j in range(0, nconf):
        for i in range(j, nconf):
            d1 = rdMolAlign.AlignMol(molrdkit_no_h, molrdkit_no_h, prbCid=i, refCid=j)
            d2 = rdMolAlign.GetBestRMS(molrdkit_no_h, molrdkit_no_h, prbId=i, refId=j)
            delt = d1 - d2
            if delt < -1e-5:
                print(f'ooops, {i}, {delt}')
            if delt > maxd:
                maxd = delt
                maxi = i
                maxj = j
    d1 = rdMolAlign.AlignMol(molrdkit_no_h, molrdkit_no_h, prbCid=maxi, refCid=maxj)
    d2 = rdMolAlign.GetBestRMS(molrdkit_no_h, molrdkit_no_h, prbId=maxi, refId=maxj)

In [8]:
    # Clusterize ========================================================================
    dists = []
    for i in range(nconf):
        for j in range(i):
            print(i, j)
            dists.append(rdMolAlign.GetBestRMS(molrdkit_no_h, molrdkit_no_h, i, j))
            # dists.append(rdMolAlign.AlignMol(molrdkit_no_h, molrdkit_no_h, i, j))
    print(dists)

    if cluster_type == "RMSD":
        clusts = Butina.ClusterData(dists, nfiles, threshold, isDistData=True, reordering=True)
    elif cluster_type == "TFD":
        tfdmat = TorsionFingerprints.GetTFDMatrix(molrdkit)
        clusts = Butina.ClusterData(tfdmat, nfiles, threshold, isDistData=True, reordering=True)
    len(clusts)

1 0
2 0
2 1
3 0
3 1
3 2
4 0
4 1
4 2
4 3
[0.8769190681038209, 0.11795360418855162, 0.8749136860279239, 0.8020211360200789, 0.6929821435742568, 0.7988247500482687, 1.4077571872750787, 1.654890450375954, 1.3612704876828092, 1.2815068876906657]


1

In [12]:
centroids = [x[0] for x in clusts]
drawit(molrdkit_no_h,centroids[:])


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

1.5369368461874025

In [10]:
drawit(molrdkit_no_h, [1, 0])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [11]:
d2 = rdMolAlign.GetBestRMS(molrdkit_no_h,molrdkit_no_h,prbId=1,refId=4)
print(d2)
drawit(molrdkit_no_h,[1,4])

1.6548904503758852


3Dmol.js failed to load for some reason. Please check your browser console for error messages.